# VQA
test accuracy=41%を超えた提出のみ，修了要件として認める

In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor

## 環境構築

*   コード
  * GitHub へ fork
  * Google Drive へ clone
  * VSCode で編集
  * Colab から Google Drive へ cd, コマンドで実行
*   学習データ
  * 自分の Google Drive に共有元フォルダへのショートカット作成
  * ショートカット経由でランタイムにコピー

### Google Drive にアクセス

In [ ]:
# Google Drive をマウント
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Final
%ls

/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Final
 configs/         'kouho 1'/        main_original.py   notebooks/          src/
 data/            'kouho 2'/        main.py            README.md           src_original/
 description.txt  'kouho 3'/        memo.txt           requirements.txt    submission.npy
 Dockerfile        main_kaizen.py   model.pth          sample_train.json   train.py


### データをランタイムにコピー, 解凍

In [ ]:
# コピー元とコピー先のディレクトリを指定
src_dir = '/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Final/data/VQA'
dst_dir = '/content/data'

# コピー先ディレクトリを作成
os.makedirs(dst_dir, exist_ok=True)

# コピーするファイルのリストを作成
files_to_copy = []
for root, _, files in os.walk(src_dir):
    for file in files:
        src_file = os.path.join(root, file)
        dst_file = os.path.join(dst_dir, os.path.relpath(src_file, src_dir))
        files_to_copy.append((src_file, dst_file))

# 並列でファイルをコピーする関数
def copy_file(src_dst):
    src_file, dst_file = src_dst
    dst_file_dir = os.path.dirname(dst_file)
    os.makedirs(dst_file_dir, exist_ok=True)
    try:
        shutil.copy2(src_file, dst_file)
    except Exception as e:
        print(f"Error copying {src_file} to {dst_file}: {e}")

# ThreadPoolExecutorを使って並列でコピー
with ThreadPoolExecutor(max_workers=24) as executor:
    executor.map(copy_file, files_to_copy)

# ファイルがすべてコピーされたかを確認
copied_files = [os.path.join(root, file) for root, _, files in os.walk(dst_dir) for file in files]
missing_files = [src for src, dst in files_to_copy if dst not in copied_files]

if missing_files:
    print(f"Missing files: {missing_files}")
else:
    print("All files copied successfully.")

All files copied successfully.


In [ ]:
# 解凍
shutil.unpack_archive('/content/data/train.zip', '/content/data')
shutil.unpack_archive('/content/data/valid.zip', '/content/data')

## 実行

In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Final'

In [ ]:
%ls

 configs/         'kouho 1'/        main_original.py   notebooks/          src/
 data/            'kouho 2'/        main.py            README.md           src_original/
 description.txt  'kouho 3'/        memo.txt           requirements.txt    submission.npy
 Dockerfile        main_kaizen.py   model.pth          sample_train.json   train.py


In [13]:
!python3 main.py

device : cuda
- START -

【1/200】
--- Training ---
train time: 505.38 [s]
train loss: 8.0832
train acc: 0.3622
train simple acc: 0.2956
--- Test ---
['black' 'blue' 'grey' 'no' 'remote control' 'unanswerable' 'white' 'yes']
test time: 201.22 [s]

【2/200】
--- Training ---
train time: 505.83 [s]
train loss: 5.3585
train acc: 0.4653
train simple acc: 0.3771
--- Test ---
['black' 'blue' 'grey' 'no' 'unanswerable' 'white' 'yes']
test time: 201.06 [s]

【3/200】
--- Training ---
train time: 504.78 [s]
train loss: 4.8313
train acc: 0.4854
train simple acc: 0.3941
--- Test ---
['black' 'black white' 'blue' 'candy' 'grey' 'keyboard' 'no' 'pink' 'red'
 'unanswerable' 'white' 'yes']
test time: 200.35 [s]

【4/200】
--- Training ---
train time: 506.09 [s]
train loss: 4.6236
train acc: 0.4948
train simple acc: 0.4032
--- Test ---
['black' 'blue' 'brown' 'green' 'grey' 'no' 'pink' 'purple' 'red'
 'unanswerable' 'white' 'yes']
test time: 199.87 [s]

【5/200】
--- Training ---
train time: 503.20 [s]
train lo

In [12]:
# 同期状況を確認
%cat main.py

import re
import random
import time
from statistics import mode

from PIL import Image
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from transformers import BertModel, BertTokenizer
import shutil
import os


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def process_text(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
   